In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/sa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [1]:
import os
import getpass

EMBEDDING_MODEL = "text-embedding-3-small"
CHAT_MODEL = "gpt-4o-mini"   # adjust as desired
COLLECTION_NAME = "adn"
TOP_K = 6
FETCH_K = 24

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta
from langchain_core.documents import Document
import csv
from typing import List

def load_csv_with_csvloader(csv_path: str) -> List[Document]:
    """
    Load a CSV into Documents using LangChain's CSVLoader.
    page_content := only ['title', 'details'].
    All remaining columns preserved as metadata (e.g., id, section, tags, channel_number).
    """
    with open(csv_path, newline="", encoding="utf-8-sig") as f:
        cols = csv.DictReader(f).fieldnames or []

    content_cols = [c for c in ["title", "details"] if c in cols]
    meta_cols = [c for c in cols if c not in content_cols]
    print(f"content_cols: {content_cols}")
    print(f"meta_cols: {meta_cols}")
    loader = CSVLoader(
        file_path=csv_path,
        content_columns=content_cols,
        metadata_columns=meta_cols,
        encoding="utf-8-sig",
        autodetect_encoding=True,
    )
    return loader.load()

# current metadata columns: id,section,subsection,title,details,price_crc,price_text,tags,url,contact_value,channel_number,locale,version
loader = CSVLoader(
    file_path=f"./RAG_data/adn_rag_base_full_v1_3.csv",
    metadata_columns=[
      "id",
      "section",
      "subsection",
      "title",
      "details",
      "price_crc",
      "price_text",
      "tags",
      "url",
      "contact_value",
      "channel_number",
      "locale",
      "version"
    ]
)



In [3]:
adn_data = load_csv_with_csvloader("RAG_data/adn_rag_base_full_v1_3.csv")

for doc in adn_data:
    print(doc.page_content)
    print(doc.metadata)
    break

content_cols: ['title', 'details']
meta_cols: ['id', 'section', 'subsection', 'price_crc', 'price_text', 'tags', 'url', 'contact_value', 'channel_number', 'locale', 'version']
title: Descripción
details: American Data Networks S.A. (ADN) fundada en 2005. Equipo con más de 15 años de experiencia en telecomunicaciones.
Enfoque 100 % en calidad de servicio y soporte oportuno. Actualización constante de tecnologías para brindar servicios de clase mundial.
Opción de transporte nacional e internacional de alta capacidad de datos en Costa Rica; permite optimizar y expandir redes de forma segura y rápida.
{'source': 'RAG_data/adn_rag_base_full_v1_3.csv', 'row': 0, 'id': '2eebd6ef-cd3e-46e4-a268-dd4830bf76aa', 'section': 'Compañía', 'subsection': 'Quiénes Somos', 'price_crc': '', 'price_text': '', 'tags': 'empresa, historia, calidad, telecomunicaciones', 'url': '', 'contact_value': '', 'channel_number': '', 'locale': 'es_CR', 'version': 'v1.3'}


In [4]:
adn_data[0]

Document(metadata={'source': 'RAG_data/adn_rag_base_full_v1_3.csv', 'row': 0, 'id': '2eebd6ef-cd3e-46e4-a268-dd4830bf76aa', 'section': 'Compañía', 'subsection': 'Quiénes Somos', 'price_crc': '', 'price_text': '', 'tags': 'empresa, historia, calidad, telecomunicaciones', 'url': '', 'contact_value': '', 'channel_number': '', 'locale': 'es_CR', 'version': 'v1.3'}, page_content='title: Descripción\ndetails: American Data Networks S.A. (ADN) fundada en 2005. Equipo con más de 15 años de experiencia en telecomunicaciones.\nEnfoque 100 % en calidad de servicio y soporte oportuno. Actualización constante de tecnologías para brindar servicios de clase mundial.\nOpción de transporte nacional e internacional de alta capacidad de datos en Costa Rica; permite optimizar y expandir redes de forma segura y rápida.')

In [5]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    adn_data,
    embeddings,
    location=":memory:",
    collection_name=COLLECTION_NAME
)

In [6]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 6})

In [66]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
Eres un asistente de soporte que trabaja para American Data Networks. Habla siempre en primera persona como si fueras parte del equipo. Responde SOLO con la información del CONTEXTO proporcionado.
Si la pregunta no se encuentra en el contexto proporcionado de American Data Networks, responde: "No tengo la respuesta a esa pregunta".
Si la pregunta es sobre un producto o servicio que no es de American Data Networks, responde: "No tenemos información sobre ese producto o servicio".

Responde en el mismo idioma en que se hizo la pregunta.
Importante: usa expresiones en primera persona (por ejemplo: “nuestra dirección”, “puedes visitarnos”, “podemos ayudarte”), nunca en tercera persona (“ellos”, “tienen”, “puedes visitarlos”).

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

In [67]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

In [68]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [69]:
naive_retrieval_chain.invoke({"question" : "¿Qué es American Data Networks y cuándo fue fundada?"})["response"].content

'American Data Networks (ADN) fue fundada en 2005. Somos un equipo con más de 15 años de experiencia en telecomunicaciones, enfocándonos en brindar servicios de calidad, soporte oportuno y actualización constante de tecnologías para ofrecer soluciones de clase mundial.'

In [70]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [71]:
from ragas.testset import TestsetGenerator
from typing import List

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)

# Create manual test set with realistic user questions
manual_test_set_updated = [
    {
        "question": "¿Qué es American Data Networks y cuándo fue fundada?",
        "contexts": [adn_data[0].page_content],
        "answer": "",
        "ground_truth": "American Data Networks S.A. (ADN) es una empresa fundada en 2005 con más de 15 años de experiencia en telecomunicaciones."
    },
    {
        "question": "¿Cuáles son los planes de Internet residencial disponibles y sus precios?",
        "contexts": [adn_data[15].page_content, adn_data[16].page_content, adn_data[17].page_content, adn_data[18].page_content],
        "answer": "",
        "ground_truth": "Los planes residenciales disponibles son: Plan 100/100 Mbps por ₡20 500 IVI, Plan 250/250 Mbps por ₡26 600 IVI, Plan 500/500 Mbps (más popular) por ₡33 410 IVI, y Plan 1/1 Gbps por ₡49 500 IVI. Todos incluyen Equipo Wi-Fi, Firewall y Soporte 24/7."
    },
    {
        "question": "¿Cómo puedo contactar a ADN para soporte técnico?",
        "contexts": [adn_data[4].page_content, adn_data[5].page_content, adn_data[7].page_content, adn_data[8].page_content],
        "answer": "",
        "ground_truth": "Puedes contactar a ADN para soporte técnico por: Teléfono principal +506 4050-5050, WhatsApp +506 7087-8240, correo helpdesk@data.cr, o soporte técnico 24/7 disponible en todo momento."
    },
    {
        "question": "¿Cuánto tiempo tarda la instalación del servicio de Internet?",
        "contexts": [adn_data[27].page_content],
        "answer": "",
        "ground_truth": "Para planes residenciales, la instalación se realiza entre 48 y 72 horas posteriores a la compra. Si la compra es antes de las 4:00 p.m., la instalación está disponible 48 horas después. Si es después de las 4:00 p.m., está disponible 72 horas después."
    },
    {
        "question": "¿Qué servicios empresariales ofrece ADN?",
        "contexts": [adn_data[33].page_content, adn_data[34].page_content, adn_data[35].page_content, adn_data[36].page_content, adn_data[37].page_content],
        "answer": "",
        "ground_truth": "ADN ofrece servicios empresariales como: Fibra Óptica para máxima velocidad y estabilidad, Data center y redes administradas con monitoreo proactivo, Telefonía VoIP con funcionalidades avanzadas, Soporte 24/7 especializado, Seguridad integral contra malware y phishing, y Firewall gestionado con soluciones personalizadas."
    }
]


rag_responses = []

for item in manual_test_set_updated:
    question = item["question"]
    
    # Usar tu pipeline RAG para generar respuesta real
    response = naive_retrieval_chain.invoke({"question": question})
    
    # Actualizar el dataset con la respuesta real del LLM
    updated_item = {
        "question": question,
        "contexts": [doc.page_content for doc in response['context']],  # Contextos reales recuperados
        "answer": response['response'].content,  # ← Respuesta real del LLM
        "ground_truth": item["ground_truth"]  # Mantener ground_truth para comparación
    }
    
    rag_responses.append(updated_item)
    
    print(f"Pregunta: {question}")
    print(f"Respuesta LLM: {response['response'].content}")
    print(f"Contextos recuperados: {len(response['context'])} documentos")
    print("-" * 50)


Pregunta: ¿Qué es American Data Networks y cuándo fue fundada?
Respuesta LLM: American Data Networks (ADN) fue fundada en 2005. Somos un equipo con más de 15 años de experiencia en telecomunicaciones, enfocados en ofrecer servicios de alta calidad y soporte oportuno, además de mantener una actualización constante en tecnologías para brindar servicios de clase mundial.
Contextos recuperados: 6 documentos
--------------------------------------------------
Pregunta: ¿Cuáles son los planes de Internet residencial disponibles y sus precios?
Respuesta LLM: Nuestros planes de Internet residencial disponibles y sus precios son los siguientes:

- Plan 100/100 Mbps: ₡20 500 IVI
- Plan 250/250 Mbps: ₡26 600 IVI
- Plan 500/500 Mbps (más popular): ₡33 410 IVI
- Plan 1/1 Gbps: ₡49 500 IVI

Todos los planes incluyen equipo Wi-Fi, firewall y soporte 24/7. Si deseas más información o ayudar con alguna de estas opciones, puedo asistirte.
Contextos recuperados: 6 documentos
------------------------------

In [74]:
# Convert to RAGAS Dataset
from datasets import Dataset
import pandas as pd

df = pd.DataFrame(rag_responses)
dataset = Dataset.from_pandas(df)

print("Test set created with 10 realistic user questions:")
print(f"Dataset shape: {df.shape}")
print("\nSample questions:")
for i, row in df.head(3).iterrows():
    print(f"{i+1}. {row['question']}")


Test set created with 10 realistic user questions:
Dataset shape: (5, 4)

Sample questions:
1. ¿Qué es American Data Networks y cuándo fue fundada?
2. ¿Cuáles son los planes de Internet residencial disponibles y sus precios?
3. ¿Cómo puedo contactar a ADN para soporte técnico?


In [75]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

# Definir las métricas que queremos evaluar
metrics = [
    faithfulness,
    answer_relevancy, 
    context_precision,
    context_recall
]

# Evaluar tu pipeline RAG
result = evaluate(
    dataset,
    metrics=metrics,
    llm=generator_llm,  # El mismo LLM que usaste para generar el test set
    embeddings=generator_embeddings  # El mismo embedding model
)

print("Resultados de la evaluación RAGAS:")
print(result)

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

Resultados de la evaluación RAGAS:
{'faithfulness': 0.8711, 'answer_relevancy': 0.9369, 'context_precision': 0.4000, 'context_recall': 0.6000}


In [80]:
import pandas as pd
import numpy as np

# Helper function to safely calculate average from RAGAS scores
def safe_average(score_value):
    """Calculate average from RAGAS score (handles both numbers and lists)."""
    if score_value is None:
        return 0.0
    elif isinstance(score_value, (int, float)):
        return float(score_value)
    elif isinstance(score_value, list):
        # Filter out None values and calculate average
        valid_scores = [s for s in score_value if s is not None and not np.isnan(s)]
        return sum(valid_scores) / len(valid_scores) if valid_scores else 0.0
    else:
        return 0.0

# Convert results to a readable table
results_dict = {
    'Metric': ['Faithfulness', 'Answer Relevancy', 'Context Precision', 'Context Recall'],
    'Score': [
        safe_average(result['faithfulness']),
        safe_average(result['answer_relevancy']), 
        safe_average(result['context_precision']),
        safe_average(result['context_recall'])
    ],
    'Description': [
        'Proportion of facts in the response that come from the context',
        'Relevance of the response to the question',
        'Precision of the retrieved context',
        'Recovery of relevant context'
    ]
}

results_df = pd.DataFrame(results_dict)
print("\nRAGAS Results Table:")
print(results_df.to_string(index=False))

# Calculate average score safely
valid_scores = [score for score in results_dict['Score'] if score > 0]
if valid_scores:
    average_score = sum(valid_scores) / len(valid_scores)
    print(f"\nRAGAS Average Score: {average_score:.4f}")
else:
    print("\nCould not calculate valid scores")

# Show individual scores for debugging - CORRECTED
print("\nIndividual Scores (Raw):")
try:
    # Access result attributes directly
    for metric in ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall']:
        try:
            score = getattr(result, metric, None)
            print(f"{metric}: {score} (type: {type(score)})")
        except AttributeError:
            # Try dictionary access
            try:
                score = result[metric]
                print(f"{metric}: {score} (type: {type(score)})")
            except (KeyError, TypeError):
                print(f"{metric}: Not available")
except Exception as e:
    print(f"Error accessing result: {e}")
    print(f"Result type: {type(result)}")
    print(f"Result dir: {dir(result)}")

# Show detailed breakdown if scores are lists - CORRECTED
print("\nDetailed Breakdown by Question:")
for metric in ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall']:
    try:
        # Try different ways to access the score
        score = None
        try:
            score = getattr(result, metric, None)
        except AttributeError:
            try:
                score = result[metric]
            except (KeyError, TypeError):
                pass
        
        if isinstance(score, list):
            print(f"\n{metric}:")
            for i, s in enumerate(score):
                print(f"  Question {i+1}: {s}")
            print(f"  Average: {safe_average(score):.4f}")
        elif score is not None:
            print(f"\n{metric}: {score}")
        else:
            print(f"\n{metric}: Not available")
    except Exception as e:
        print(f"\n{metric}: Error - {e}")


RAGAS Results Table:
           Metric    Score                                                    Description
     Faithfulness 0.871111 Proportion of facts in the response that come from the context
 Answer Relevancy 0.936899                      Relevance of the response to the question
Context Precision 0.400000                             Precision of the retrieved context
   Context Recall 0.600000                                   Recovery of relevant context

RAGAS Average Score: 0.7020

Individual Scores (Raw):
faithfulness: None (type: <class 'NoneType'>)
answer_relevancy: None (type: <class 'NoneType'>)
context_precision: None (type: <class 'NoneType'>)
context_recall: None (type: <class 'NoneType'>)

Detailed Breakdown by Question:

faithfulness: Not available

answer_relevancy: Not available

context_precision: Not available

context_recall: Not available
